# DGGS grid conversions
This notebook introduces and demonstrates conversions of spatial data into two prominent Discrete Global Grid Systems (DGGS): H3 and HEALPix, using the PixCDust library.

DGGS are spatial reference systems that partition the globe into hierarchical grids. These grids divide the Earth into regions that can be easily indexed, offering advantages for spatial analysis, data integration, and visualization. DGGS are increasingly used in applications like environmental monitoring, geospatial analysis, and big data processing, because they provide a standardized way to manage spatial data across different scales.

In [1]:
# imports
from pixcdust.downloaders.hydroweb_next import PixCDownloader
from pixcdust.readers import NcSimpleReader
import geopandas as gpd
from datetime import datetime
import xarray as xr
import glob

### Download

In [2]:
# reading the area of interest
gdf_geom = gpd.read_file("../data/aoi.gpkg")

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=1,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

Downloaded products:   0%|                                                                                    …

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

### Reading nc files and reprojecting into dggs grids

In [4]:
# Search swot nc files
swot_nc_files = glob.glob('/tmp/pixc'+'/*/*nc')
print(swot_nc_files)

['/tmp/pixc/SWOT_L2_HR_PIXC_483_016_078L_20230407T093656_20230407T093707_PGC0_01_swot_d476ce69-7756-3de7-99b7-4c70671461cb_V1-SWOT_L2_HR_PIXC_483_016_078L_20230407T093656_20230407T093707_PGC0_01_swot_d476ce69-7756-3de7-99b7-4c70671461cb_V1/SWOT_L2_HR_PIXC_483_016_078L_20230407T093656_20230407T093707_PGC0_01.nc', '/tmp/pixc/SWOT_L2_HR_PIXC_482_016_078L_20230406T094618_20230406T094629_PGC0_01_swot_b067e476-1232-363f-9ca8-2ff9791fa7b3_V1-SWOT_L2_HR_PIXC_482_016_078L_20230406T094618_20230406T094629_PGC0_01_swot_b067e476-1232-363f-9ca8-2ff9791fa7b3_V1/SWOT_L2_HR_PIXC_482_016_078L_20230406T094618_20230406T094629_PGC0_01.nc']


In [5]:
# Read one swot nc file
path = swot_nc_files[0]
reader = NcSimpleReader(path)
reader.read()

# Print variables
reader.data.data_vars

Data variables:
    azimuth_index                          (points) float64 49MB ...
    range_index                            (points) float64 49MB ...
    interferogram                          (points, complex_depth) float32 49MB ...
    power_plus_y                           (points) float32 25MB ...
    power_minus_y                          (points) float32 25MB ...
    coherent_power                         (points) float32 25MB ...
    x_factor_plus_y                        (points) float32 25MB ...
    x_factor_minus_y                       (points) float32 25MB ...
    water_frac                             (points) float32 25MB ...
    water_frac_uncert                      (points) float32 25MB ...
    classification                         (points) float32 25MB ...
    false_detection_rate                   (points) float32 25MB ...
    missed_detection_rate                  (points) float32 25MB ...
    prior_water_prob                       (points) float32 25MB ...
   

In [6]:
# Chose one or more variables
reader.data['height']

<xarray.DataArray 'height' (points: 6137280)> Size: 25MB
[6137280 values with dtype=float32]
Coordinates:
    latitude   (points) float64 49MB 43.77 43.76 43.77 ... 43.35 43.35 43.35
    longitude  (points) float64 49MB 0.7255 0.7249 0.7254 ... 1.728 1.727 1.728
  * points     (points) object 49MB POINT (0.7255257239487491 43.765118151867...
Indexes:
    points   GeometryIndex (crs=EPSG:4326)
Attributes:
    long_name:     height above reference ellipsoid
    units:         m
    quality_flag:  geolocation_qual
    valid_min:     -1500.0
    valid_max:     15000.0
    comment:       Height of the pixel above the reference ellipsoid.

In [7]:
# Reproject variables into h3 grid
ds_h3 = reader.to_h3(variables = 'height', resolution = 8) # Modify the resolution if needeed
ds_h3

<xarray.Dataset> Size: 178kB
Dimensions:   (cell_ids: 6351)
Coordinates:
  * cell_ids  (cell_ids) int64 51kB 613498900998782975 ... 613499360254099455
    h3_lon    (cell_ids) float64 51kB 1.613 1.624 1.606 ... 0.7561 0.7668 0.7491
    h3_lat    (cell_ids) float64 51kB 43.58 43.58 43.58 ... 43.72 43.71 43.71
Data variables:
    height    (cell_ids) float32 25kB 236.5 240.1 285.0 ... 265.5 250.5 254.3
Indexes:
    cell_ids  H3Index(level=8)
Attributes:
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3244
    interferogram_size_range:    4856
    looks_to_efflooks:           1.5337356015127774
    num_azimuth_looks:           7.0
    azimuth_offset:              6

In [8]:
# Show
ds_h3['height'].dggs.explore()

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [9]:
# Reproject variables into healpix grid
ds_healpix = reader.to_healpix(variables = 'height', resolution = 13) # modify resolution if needed
ds_healpix

<xarray.Dataset> Size: 197kB
Dimensions:      (cell_ids: 7047)
Coordinates:
  * cell_ids     (cell_ids) int64 56kB 123386469 123386471 ... 126962196
    healpix_lon  (cell_ids) float64 56kB 1.484 1.507 1.518 ... 0.9431 0.9544
    healpix_lat  (cell_ids) float64 56kB 43.91 43.91 43.91 ... 43.21 43.21 43.21
Data variables:
    height       (cell_ids) float32 28kB 238.4 213.9 244.6 ... 426.7 434.2 412.5
Indexes:
    cell_ids  HealpixIndex(nside=13, indexing_scheme=ring)
Attributes:
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3244
    interferogram_size_range:    4856
    looks_to_efflooks:           1.5337356015127774
    num_azimuth_looks:           7.0
    azimuth_offset:              6

In [10]:
# Show
ds_healpix['height'].dggs.explore()

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

Enjoy !